In [ ]:
import unicodedata
import json
import re
from datetime import datetime
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine


# Helper function to load JSON data
def load_json(file_path):
    """Load a JSON file and return the data."""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None


# Extract text parts from a message
def extract_message_parts(message):
    content = message.get("content")
    if content and content.get("content_type") == "text":
        return content.get("parts", [])
    return []


# Get the author name
def get_author_name(message):
    author = message.get("author", {}).get("role", "")
    if author == "assistant":
        return "ChatGPT"
    elif author == "system":
        return "Custom user info"
    return author


# Extract messages from a conversation
def get_conversation_messages(conversation):
    messages = []
    current_node = conversation.get("current_node")
    mapping = conversation.get("mapping", {})
    while current_node:
        node = mapping.get(current_node, {})
        message = node.get("message") if node else None
        if message:
            parts = extract_message_parts(message)
            author = get_author_name(message)
            if parts and len(parts) > 0 and len(parts[0]) > 0:
                if author != "system" or message.get("metadata", {}).get("is_user_system_message"):
                    create_time = message.get("create_time")
                    update_time = message.get("update_time")

                    # Handle None values for timestamps
                    if create_time is None:
                        create_time = 0  # Set to 0 as a fallback (Unix epoch)
                    if update_time is None:
                        update_time = 0  # Set to 0 as a fallback

                    messages.append({
                        "author": author,
                        "text": parts[0],
                        "create_time": datetime.fromtimestamp(create_time),
                        "update_time": datetime.fromtimestamp(update_time)
                    })
        current_node = node.get("parent") if node else None
    return messages[::-1]


# Create a directory based on date
def create_directory(base_dir, date):
    directory_name = date.strftime("%Y_%m")
    directory_path = base_dir / directory_name
    directory_path.mkdir(parents=True, exist_ok=True)
    return directory_path


# Sanitize the title for valid file names
def sanitize_title(title):
    title = unicodedata.normalize("NFKC", title)
    title = re.sub(r'[<>:"/\\|?*\x00-\x1F\s]', '_', title)
    return title[:140]


# Create file name for conversation
def create_file_name(directory_path, title, date):
    sanitized_title = sanitize_title(title)
    return directory_path / f"{date.strftime('%Y_%m_%d')}_{sanitized_title}.txt"


# Write messages to text file
def write_messages_to_file(file_path, messages):
    with file_path.open("w", encoding="utf-8") as file:
        for message in messages:
            file.write(f"{message['author']}\n")
            file.write(f"{message['text']}\n")


# Extract data into DataFrames
def extract_conversations_to_df(conversations_data):
    conversation_records = []
    message_records = []

    for conversation in conversations_data:
        updated = conversation.get("update_time")
        if not updated:
            continue

        updated_date = datetime.fromtimestamp(updated) if updated else datetime.fromtimestamp(0)
        title = conversation.get("title", "Untitled")

        # Add conversation-level data
        conversation_records.append({
            "conversation_id": conversation.get("id", "Unknown"),
            "title": title,
            "create_time": datetime.fromtimestamp(conversation.get("create_time", 0)),
            "update_time": updated_date
        })

        # Extract messages
        messages = get_conversation_messages(conversation)
        for message in messages:
            message_records.append({
                "conversation_id": conversation.get("id", "Unknown"),
                "author": message['author'],
                "text": message['text'],
                "create_time": message['create_time'],
                "update_time": message['update_time']
            })

    conversations_df = pd.DataFrame(conversation_records)
    messages_df = pd.DataFrame(message_records)

    return conversations_df, messages_df


# Save DataFrames to a SQLite database
def save_to_database(conversations_df, messages_df, db_uri='sqlite:///conversations.db'):
    engine = create_engine(db_uri)
    with engine.connect() as conn:
        conversations_df.to_sql("conversations", con=conn, if_exists='replace', index=False)
        messages_df.to_sql("messages", con=conn, if_exists='replace', index=False)


# Main function to process conversations and save data
def process_conversations(input_file, output_dir, db_uri='sqlite:///conversations.db'):
    input_file = Path(input_file)
    output_dir = Path(output_dir)

    with input_file.open("r", encoding="utf-8") as file:
        conversations_data = json.load(file)

    # Extract DataFrames
    conversations_df, messages_df = extract_conversations_to_df(conversations_data)

    # Save data to SQLite database
    save_to_database(conversations_df, messages_df, db_uri)

    # Save messages to text files
    created_directories_info = []
    for _, conversation in conversations_df.iterrows():
        updated_date = conversation['update_time']
        directory_path = create_directory(output_dir, updated_date)
        file_name = create_file_name(directory_path, conversation['title'], updated_date)

        # Filter messages for this conversation
        conv_messages = messages_df[messages_df['conversation_id'] == conversation['conversation_id']].to_dict('records')

        # Write messages to file
        write_messages_to_file(file_name, conv_messages)

        created_directories_info.append({"directory": str(directory_path), "file": str(file_name)})

    return created_directories_info


# Example to run the function locally in Jupyter or as a script
def run_process(input_file='conversations.json', output_dir='output_directory'):
    """
    Run the process to extract data and save to files and database.
    """
    created_directories_info = process_conversations(input_file, output_dir)

    for info in created_directories_info:
        print(f"Created {info['file']} in directory {info['directory']}")


# You can call this function in a Jupyter Notebook or run it as a standalone script
# Example: 'conversations.json' should be replaced with the actual path
run_process('conversations.json', 'ChatGPT_Convos')
